In [1]:
import json
from textwrap import dedent
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI
from datasets import load_dataset

In [3]:
client = OpenAI()

In [2]:
mmlu = load_dataset('lukaemon/mmlu', 'elementary_mathematics')
ds = mmlu['train']

In [ ]:
system_prompt = {
    'role': 'system',
    'content': dedent('''
        You are an helpful assistant who produces synthetic data in the subject of {subject}.
        Always reply with question and 4 choices for the answers.
        Generate a new question in the similar fashion to what has been shared.
        You always output in JSON format.
    ''').strip().format(subject='elementary mathematics')
}

example_prompt = dedent('''
    Question: {question}

    Choices:
        A: {option_a}
        B: {option_b}
        C: {option_c}
        D: {option_d}
''').strip()

In [ ]:
ds[0]

In [ ]:
user_prompt = {
    'role': 'user',
    'content': example_prompt.format(
        question=ds[0]['input'],
        option_a=ds[0]['A'],
        option_b=ds[0]['B'],
        option_c=ds[0]['C'],
        option_d=ds[0]['D'],
    )
}

assistant_prompt = {
    'role': 'assistant',
    'content': '''
    {
    "Question": "Olivia used the rule 'Add 11' to create the number pattern shown below. 10, 21, 32, 43, 54. Which statement about the number pattern is true?",
    "Choices": {
        "A": "The 10th number in the pattern will be an even number.",
        "B": "The number pattern will never have two even numbers next to each other.",
        "C": "The next two numbers in the pattern will be an even number then an odd number.",
        "D": "If the number pattern started with an odd number then the pattern would have only odd numbers in it."
    }
    }
    '''
}

user_prompt1 = {
    'role': 'user',
    'content': example_prompt.format(
        question=ds[1]['input'],
        option_a=ds[1]['A'],
        option_b=ds[1]['B'],
        option_c=ds[1]['C'],
        option_d=ds[1]['D'],
    )
}

In [ ]:
try:
    completions = client.chat.completions.create(
        model='gpt-3.5-turbo-1106',
        response_format={'type': 'json_object'},
        messages=[
            system_prompt,
            # user_prompt,
            # assistant_prompt,
            user_prompt1
        ]
    )

    if completions.choices[0].finish_reason == 'length':
        raise IOError('Reached maximum output length')

    try:
        op = json.loads(completions.choices[0].message.content)
    except:
        raise ValueError('Value returned by the model is not valid JSON')

    print(f'Tokens used" {completions.usage}')

except Exception as err:
    print(f'Error raised in accesing the API: {err}')

In [ ]:
print(json.dumps(op, indent=4))

In [ ]:
# parse into pydantic class (so that we can verify)
# and add to responses list

### Hindi articles generation

In [4]:
system_prompt = {
    'role': 'system',
    'content': dedent('''
        You are an helpful assistant who produces synthetic data in colloquial Devnagri Hindi.
        The data should be relevant to India.
    ''').strip()
}

user_prompt = {
    'role': 'user',
    'content': 'Generate a fictional story set in a school classroom (200-600 words)'
}

In [5]:
try:
    completions = client.chat.completions.create(
        model='gpt-4-1106-preview',
        # response_format={'type': 'json_object'},
        messages=[
            system_prompt,
            user_prompt,
        ]
    )

    if completions.choices[0].finish_reason == 'length':
        raise IOError('Reached maximum output length')

    try:
        op = json.loads(completions.choices[0].message.content)
    except:
        raise ValueError('Value returned by the model is not valid JSON')

    print(f'Tokens used" {completions.usage}')

except Exception as err:
    print(f'Error raised in accesing the API: {err}')

Error raised in accesing the API: Value returned by the model is not valid JSON


In [6]:
print(completions.usage)

CompletionUsage(completion_tokens=1723, prompt_tokens=53, total_tokens=1776)


In [7]:
print(completions.choices[0].message.content)

एक बार की बात है, मध्य प्रदेश के एक छोटे से गाँव के सरकारी स्कूल में एक नई अध्यापिका आईं। उनका नाम था नीति मैम। उन्होंने गाँव के बच्चों के मन में विज्ञान की दिलचस्पी जगाने की ठान ली।

पहले दिन स्कूल में नीति मैम ने बच्चों से परिचय कराया। वे बच्चों से बोलीं, "देखो बच्चों, विज्ञान कोई जटिल विषय नहीं है। यह तो हमारे चारों ओर है।" फिर उन्होंने अपनी जेब से एक छोटा सा आईना निकाला और धूप में इस तरह घुमाया कि एक बच्चे के चेहरे पर रौशनी की एक छोटी सी बिंदी दिखाई दे। तपाक से एक बच्चे ने पूछा, "मैम ये कैसे हुआ?"

नीति मैम मुस्कुराईं और बोलीं, "यह प्रकाश के परावर्तन का उदाहरण है, जो विज्ञान का एक हिस्सा है।" उनके इस सरल उदाहरण से बच्चों की समझ में आने लगा कि विज्ञान वास्तव में उनके आस-पास ही है।

उस दिन से, हर सुबह बच्चे उत्सुकता से स्कूल आते क्योंकि नीति मैम का पढ़ाया हर सबक एक नया अविष्कार लगता। वे खेल-खेल में विज्ञान की जटिल बातों को आसान तरीके से समझातीं।

एक दिन उन्होंने बच्चों से कहा, "आज हम मौसम के बारे में जानेंगे।" उन्होंने बच्चों को बाहर ले जाकर आसमान में बादलों को दिखाया और बच्चों को सम

~1800 tokens = 90 seconds